Немного полезных функций

Загрузка данных из файла. Тут две полезные конструкции. 
Первая, это построчное чтение реализованное с помощью
~~~
for line in open(path+'/movies.dat'):
~~~
Вторая -- функция split(), позволяющая разделить строку, по символу или последовательности символов в качестве разделителя.


In [28]:
#Чтение данных

def loadMovieLens1M(path='./ml-1m'):
# Получить названия фильмов
    movies={}
    for line in open(path+'/movies.dat'):
        (id,title)=line.split('::')[0:2] #при желании тут можно добавить чтение жанров.
        movies[id]=title
# Загрузить данные
    prefs={}
    for line in open(path+'/ratings.dat'):
        (user,movieid,rating,ts)=line.split('::')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
    return prefs

In [29]:
%%time
critics = loadMovieLens1M()

Wall time: 775 ms


Тестовая и обучающая выборки.
Рассмотрим выбор или не выбор элемента с заданной вероятностью, для этого нам понадобится модуль random и функция random(). Последняя возвращает случайное число равномерно распределенное в диапазоне от 0 до 1. Этим удобно пользоваться, чтобы получить случайное событие с вероятностью \alpha достаточно использовать конструкцию 
~~~
if random()<alpha:
~~~

Функция seed() инициализирует генератор псевдослучайных чисел заданным числом. Это позволяет воспроизводить идентичную последовательность случайных значений при последовательных запусков. Очень полезно при отладке.

In [8]:
from random import random, seed
seed(100)
print(random(), random(), random())
print(random(), random(), random())
seed(100)
print(random(), random(), random())


0.1456692551041303 0.45492700451402135 0.7707838056590222
0.705513226934028 0.7319589730332557 0.43351443489540376
0.1456692551041303 0.45492700451402135 0.7707838056590222


Отберем, например, около 2 процентов пользователей:

In [10]:
seed(20)
user_list = [u for u in critics if random()<0.02]

In [14]:
len(user_list)

131

In [12]:
sub_movies = {}
for u in user_list:
    for item in critics[u]:
        if not item in sub_movies:
            sub_movies[item] = 0
        sub_movies[item] += 1
            

In [13]:
sub_movies

{'Who Framed Roger Rabbit? (1988)': 45,
 'Honey, I Shrunk the Kids (1989)': 26,
 'Treasure of the Sierra Madre, The (1948)': 9,
 'Dragonheart (1996)': 18,
 'Jumanji (1995)': 17,
 'Lifeboat (1944)': 12,
 'Great Escape, The (1963)': 20,
 'Rock, The (1996)': 34,
 'Star Wars: Episode V - The Empire Strikes Back (1980)': 73,
 'Dances with Wolves (1990)': 34,
 'Princess Bride, The (1987)': 51,
 'Twister (1996)': 22,
 'Red Dawn (1984)': 10,
 'Raiders of the Lost Ark (1981)': 62,
 'Batman (1989)': 36,
 'Force 10 from Navarone (1978)': 9,
 'Quick and the Dead, The (1995)': 6,
 'Crimson Tide (1995)': 22,
 'Patton (1970)': 18,
 'Barbarella (1968)': 11,
 'All Quiet on the Western Front (1930)': 7,
 'Judge Dredd (1995)': 17,
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)': 34,
 'Popeye (1980)': 19,
 'Ben-Hur (1959)': 19,
 'Lord of the Flies (1963)': 10,
 'War of the Worlds, The (1953)': 12,
 'Operation Dumbo Drop (1995)': 6,
 'Pork Chop Hill (1959)': 4,
 'Star Trek: G

Оставим только фильмы которые оценило хотя бы 5 человек

In [15]:
movie5 = [m for m in sub_movies if sub_movies[m]>4]

In [16]:
len(movie5)

1559

In [17]:
movie5_sublist = [m for m in movie5 if random()<0.1]

In [18]:
len(movie5_sublist)

167

In [19]:
sub_critics = {}
for u in user_list:
    sub_critics[u] = {}
    for m in critics[u]:
        if m in movie5_sublist:
            sub_critics[u][m] = critics[u][m]

In [20]:
sub_critics

{'40': {'Jumanji (1995)': 5.0,
  'Great Escape, The (1963)': 5.0,
  'Force 10 from Navarone (1978)': 2.0,
  'Ben-Hur (1959)': 5.0,
  'Lord of the Flies (1963)': 2.0,
  'Operation Dumbo Drop (1995)': 3.0,
  'M*A*S*H (1970)': 4.0,
  'Aces: Iron Eagle III (1992)': 1.0,
  'Wizard of Oz, The (1939)': 5.0,
  'Black Beauty (1994)': 2.0,
  'Romancing the Stone (1984)': 4.0,
  'Tron (1982)': 3.0},
 '66': {'Rain Man (1988)': 5.0},
 '89': {'Naked Gun 2 1/2: The Smell of Fear, The (1991)': 1.0,
  'Good Will Hunting (1997)': 4.0,
  'Rain Man (1988)': 4.0,
  'Airplane II: The Sequel (1982)': 1.0},
 '106': {'Little Mermaid, The (1989)': 5.0,
  'Rebecca (1940)': 4.0,
  'Breakfast Club, The (1985)': 4.0},
 '173': {'Thunderball (1965)': 4.0,
  'Duck Soup (1933)': 5.0,
  'Atlantic City (1980)': 4.0,
  'Heat (1995)': 5.0,
  'Great Escape, The (1963)': 5.0,
  'Contender, The (2000)': 2.0,
  'Ben-Hur (1959)': 4.0,
  'Producers, The (1968)': 4.0,
  'M*A*S*H (1970)': 5.0,
  'Rope (1948)': 4.0,
  'Wizard of Oz

попробуем простейший вариант стохастического градиентного спуска

In [32]:
seed(13)
L = 5
Pi = {}
Bi = {}
for u in user_list:
    Bi[u] = 0
    Pi[u] = []
    for i in range(L):
        Pi[u].append(random())
Ba = {}
Qa = {}        
for m in movie5_sublist:
    Ba[m] = 0 
    Qa[m] = []
    for i in range(L):
        Qa[m].append(random())


In [22]:
rating_list = []
for u in sub_critics:
    for m in sub_critics[u]:
        rating_list.append((u,m))

In [23]:
len(rating_list)

2234

In [24]:
def evaluate_ratings(rating_list, critics, Pi, Qa, Bi, Ba, L = 5):
    summ = 0
    for (u,m) in rating_list:
        rate = Bi[u] + Ba[m]
        for i in range(L):
            rate += Pi[u][i]*Qa[m][i]
        summ += (critics[u][m] - rate)*(critics[u][m] - rate)
    return summ
        

In [38]:
evaluate_ratings(rating_list, sub_critics, Pi, Qa, Bi, Ba, L = 5)

16295.542306707712

In [41]:
from math import sqrt
def error_for_user(rating_list, sub_critics, Pi, Qa, Bi, Ba, L = 5):
    ev_rat = evaluate_ratings(rating_list, sub_critics, Pi, Qa, Bi, Ba, L = 5)
    error = sqrt(ev_rat / len(rating_list))
    return(error)
error_for_user(rating_list, sub_critics, Pi, Qa, Bi, Ba, L = 5)

2.700802485829152

$E = \sum(R(u, m) - Bi[u] - Ba[m] -Qa[m]**T * Pi[u])^2$
u = user
m = movie
R(u, m) = real rating
Bi[u] = feature of user
Ba[m] = feature of movie

In [51]:
gamma = 0.01

In [55]:
%%time

for jjj in range(10):
    for (u, m) in rating_list:
    #one step of gradient
        rate = Bi[u] + Ba[m]
        for i in range(L):
            rate += Pi[u][i]*Qa[m][i]
        error = rate - sub_critics[u][m]
        deltaBi = gamma*error     
        deltaBa = gamma*error     
        Bi[u] -= deltaBi
        Ba[m] -= deltaBi
        for i in range(L):
            deltaPi = (gamma*error*Qa[m][i])
            deltaQa = (gamma*error*Pi[u][i])
            Pi[u][i] -= deltaPi
            Qa[m][i] -= deltaQa
    print(evaluate_ratings(rating_list, sub_critics, Pi, Qa, Bi, Ba))
    print(error_for_user(rating_list, sub_critics, Pi, Qa, Bi, Ba))

280.22863899159586
0.35417236735450075
280.2010887574381
0.35415495697930893
280.1735853639503
0.35413757535132634
280.1461284881024
0.3541202222706221
280.1187178099032
0.35410289753914004
280.091353012363
0.3540856009606754
280.0640337814495
0.35406833234084795
280.03675980606005
0.35405109148708475
280.0095307779703
0.3540338782085886
279.98234639181254
0.35401669231632443
Wall time: 158 ms


In [31]:
evaluate_ratings(rating_list, sub_critics, Pi, Qa, Bi, Ba, L = 5)

834.4861444758175

{'40': [0.647505223807018,
  0.9359967840634172,
  0.8748491523896436,
  0.8678558526453876,
  0.2663735560283994],
 '66': [0.8110242089556595,
  0.6546135772776358,
  0.6904258351332923,
  0.5032677904332925,
  0.1075072747623834],
 '89': [1.0000608041117214,
  0.92263983001497,
  0.5595391897106834,
  0.8814594773412114,
  0.354843033602531],
 '106': [0.946058553839944,
  0.14087574237680656,
  0.6136226743520912,
  0.982713722834086,
  0.960502878234194],
 '173': [0.7766117775164072,
  1.0192898348008885,
  0.6986474998509903,
  0.987014342452625,
  0.304779894128943],
 '323': [0.7927030582708133,
  0.5786915263906355,
  0.8660751049555315,
  0.5395824059393519,
  0.2773167638749264],
 '331': [0.754665904677279,
  0.3964540213631376,
  0.21282530719269604,
  0.9482006719012869,
  1.010537414200811],
 '366': [0.9799074165600458,
  0.7695499155334989,
  0.4481696522822446,
  0.21238012142307824,
  0.7926230004077269],
 '413': [0.7528918373719847,
  0.30556456566997614,
  0.67725921300